In [ ]:
# Import necessary libraries

In [1]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

In [81]:

from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=5):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
# download bird and forest pictures, duckduckgo api not working

In [82]:
UNSPLASH_ACCESS_KEY="_cCqPIjHpijlT288W1p94bk_oInij8VCArlZK2359PM"

In [91]:
import os
from os.path import join, dirname, abspath
from dotenv import load_dotenv

dotenv_path = join(dirname(abspath("__file__")), '../.env')
load_dotenv(dotenv_path)

UNSPLASH_ACCESS_KEY = os.environ.get("UNSPLASH_ACCESS_KEY")

In [ ]:
# use flickrapi scape and save photos to local folder

In [116]:
import flickrapi

# Set your API key and secret
api_key = "b1b406f4edd7ae93e6ae77fd691baed8"
api_secret = "a99a6dac4227bcab"

import os
import urllib.request
import flickrapi


# Create the FlickrAPI object
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Set the search terms and categories
search_terms = {
    "forest_photo": "forest",
    "forest_sun_photo": "forest sun",
    "forest_shade_photo": "forest shade",
    "bird_sun_photo": "bird sun",
    "bird_shade_photo": "bird shade"
}

# Set the number of photos to download per category
num_photos_per_category = 20

# Download photos for each category and save them to separate folders
for category, search_term in search_terms.items():
    # Search for photos and get the download URLs
    photos = flickr.photos.search(text=search_term, per_page=num_photos_per_category, media='photos', tag_mode='all', tags=search_term)
    photo_urls = []
    for photo in photos['photos']['photo']:
        photo_id = photo['id']
        photo_info = flickr.photos.getInfo(photo_id=photo_id, format='parsed-json')
        photo_url = f"https://live.staticflickr.com/{photo_info['photo']['server']}/{photo_id}_{photo_info['photo']['secret']}_b.jpg"
        photo_urls.append(photo_url)

    # Create the local folder for the photos
    folder_name = f"saved_photos/{category}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Download the photos and save them to the local folder
    for i, url in enumerate(photo_urls):
        local_filename = f"{folder_name}/{category}_{i}.jpg"
        urllib.request.urlretrieve(url, local_filename)
        print(f"Saved photo {i+1}/{num_photos_per_category} to {local_filename}")

Saved photo 1/20 to saved_photos/forest_photo/forest_photo_0.jpg
Saved photo 2/20 to saved_photos/forest_photo/forest_photo_1.jpg
Saved photo 3/20 to saved_photos/forest_photo/forest_photo_2.jpg
Saved photo 4/20 to saved_photos/forest_photo/forest_photo_3.jpg
Saved photo 5/20 to saved_photos/forest_photo/forest_photo_4.jpg
Saved photo 6/20 to saved_photos/forest_photo/forest_photo_5.jpg
Saved photo 7/20 to saved_photos/forest_photo/forest_photo_6.jpg
Saved photo 8/20 to saved_photos/forest_photo/forest_photo_7.jpg
Saved photo 9/20 to saved_photos/forest_photo/forest_photo_8.jpg
Saved photo 10/20 to saved_photos/forest_photo/forest_photo_9.jpg
Saved photo 11/20 to saved_photos/forest_photo/forest_photo_10.jpg
Saved photo 12/20 to saved_photos/forest_photo/forest_photo_11.jpg
Saved photo 13/20 to saved_photos/forest_photo/forest_photo_12.jpg
Saved photo 14/20 to saved_photos/forest_photo/forest_photo_13.jpg
Saved photo 15/20 to saved_photos/forest_photo/forest_photo_14.jpg
Saved photo 16

In [ ]:
# install and import fastai

In [45]:
import fastai
from fastai.learner import Learner
from fastai.vision.core import *
from fastai.vision.utils import verify_images
from pathlib import Path
from fastai.vision.all import *


# Set the path to your image directory
path = Path("/Users/wukaiti/deeplearning/saved_photos")

# Get a list of all image files in the directory
image_files = get_image_files(path)

# Verify the integrity of each image file
verify_images(image_files)
failed = verify_images(get_image_files("/Users/wukaiti/deeplearning/saved_photos"))
failed.map(Path.unlink)
len(failed)

0

In [ ]:
# train images

In [46]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
# show results

In [47]:
blocks=(ImageBlock, CategoryBlock),
get_items=get_image_files, 
splitter=RandomSplitter(valid_pct=0.2, seed=42),
get_y=parent_label,
item_tfms=[Resize(192, method='squish')]
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

epoch,train_loss,valid_loss,error_rate,time
0,2.606458,3.972692,0.933333,00:01


epoch,train_loss,valid_loss,error_rate,time
0,2.579088,3.196482,0.933333,00:01
1,2.574564,2.651485,0.933333,00:01
2,2.300069,2.102962,0.933333,00:01


In [ ]:
# try model on one photo

In [48]:
is_bird,_,probs = learn.predict(PILImage.create('/Users/wukaiti/deeplearning/saved_photos/bird_29.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

This is a: saved_photos.
Probability it's a bird: 0.0251
